# Notatki do pracowni drugiej

Rozwiązujemy układ równań

\begin{equation}
(x-x_1)^2 + (y-y_1)^2 + (z-z_1)^2 = \big[c(t_1-t)\big]^2 \\
(x-x_2)^2 + (y-y_2)^2 + (z-z_2)^2 = \big[c(t_2-t)\big]^2 \\
(x-x_3)^2 + (y-y_3)^2 + (z-z_3)^2 = \big[c(t_3-t)\big]^2 \\
(x-x_4)^2 + (y-y_4)^2 + (z-z_4)^2 = \big[c(t_4-t)\big]^2 \\
\end{equation}

Niech
\begin{equation}
f_i(x, y, z, t) = (x-x_i)^2 + (y-y_i)^2 + (z-z_i)^2 - \big[c(t_i-t)\big]^2
\end{equation}

Z uogólnionego twierdzenia Taylora mamy
\begin{equation}
f_i(x + h_x, y + h_y, z + h_z, t + h_t) \approx f_i(x, y, z, t) + h_x \frac{\partial f_i}{\partial x}(x, y, z, t) 
+ h_y \frac{\partial f_i}{\partial y}(x, y, z, t) + h_z \frac{\partial f_i}{\partial z}(x, y, z, t)
+ h_t \frac{\partial f_i}{\partial t}(x, y, z, t)
\end{equation}

Ponadto
\begin{equation}
\frac{\partial f_i}{\partial x} = 2(x-x_i) \\
\frac{\partial f_i}{\partial y} = 2(y-y_i) \\
\frac{\partial f_i}{\partial z} = 2(z-z_i) \\
\frac{\partial f_i}{\partial t} = -2c^2(t-t_i)
\end{equation}

Stąd
\begin{equation}
f_i(x + h_x, y + h_y, z + h_z, t + h_t) \approx f_i(x, y, z, t) + 2h_x(x-x_i) + 2h_y(y-y_i) + 2h_z(z-z_i) -2c^2h_t(t-t_i)
\end{equation}

## Metoda Newtona

Zastosujemy metodę Newtona. Chcemy znaleźć $x, y, z, t$, dla których $f_i(x, y, z, t) = 0$ dla $1 \leq i \leq 4$.
Niech $x_n \in \mathbb{R}^4$ będzie n-tym przybliżeniem metody Newtona. Szukamy $h \in \mathbb{R}^4$, takiego że
dla $x_{n+1} = x_n + h$ zajdzie $f_i(x_{n+1}) = 0$ dla $1 \leq i \leq 4$.

Niech
$$
x_n = 
    \begin{pmatrix}
    x \\
    y \\
    z \\
    t
    \end{pmatrix}
$$

Rozwiążemy układ równań
\begin{equation}
0 = f_1(x, y, z, t) + 2h_x(x-x_1) + 2h_y(y-y_1) + 2h_z(z-z_1) -2c^2h_t(t-t_1) \\
0 = f_2(x, y, z, t) + 2h_x(x-x_2) + 2h_y(y-y_2) + 2h_z(z-z_2) -2c^2h_t(t-t_2) \\
0 = f_3(x, y, z, t) + 2h_x(x-x_3) + 2h_y(y-y_3) + 2h_z(z-z_3) -2c^2h_t(t-t_3) \\
0 = f_4(x, y, z, t) + 2h_x(x-x_4) + 2h_y(y-y_4) + 2h_z(z-z_4) -2c^2h_t(t-t_4)
\end{equation}
Inaczej
\begin{equation}
\begin{pmatrix}
x-x_1  & y-y_1 & z-z_1 & -c^2(t-t_1) \\
x-x_2  & y-y_2 & z-z_2 & -c^2(t-t_2) \\
x-x_3  & y-y_3 & z-z_3 & -c^2(t-t_3) \\
x-x_4  & y-y_4 & z-z_4 & -c^2(t-t_4) \\
\end{pmatrix}
\begin{pmatrix}
h_x \\ h_y \\ h_z \\ h_t
\end{pmatrix}
=
-\frac{1}{2}
\begin{pmatrix}
f_1(x, y, z, t) \\
f_2(x, y, z, t) \\
f_3(x, y, z, t) \\
f_4(x, y, z, t)
\end{pmatrix}
\end{equation}

Wtedy 
$$
x_{n+1} = x_n + h = x_n + \begin{pmatrix} h_x \\ h_y \\ h_z \\ h_t \end{pmatrix}
$$

In [108]:
function fsingle(x, sat)
    c = 299792.458
    a = (x - sat) .^ 2
    a[4] *= -c^2
    sum(a)
end 

function fvect(x, sats)
    [fsingle(x, sats[i]) for i = 1:length(sats)]
end

function getcoeffs(x, sats)
    c = 299792.458
    A = [x - sats[i] for i = 1:length(sats)]
    A = hcat(A...)'
    A[:, 4] *= -c^2
    A
end

function newton(sats; maxiter::Int64=20) 
    c = 299792.458
    x = zeros(4)
    for i = 1:maxiter
        A = getcoeffs(x, sats)
        B = fvect(x, sats)
        h = -0.5 * (A \ B)
        x = x + h
    end
    x
end;

In [109]:
using Polynomials

function algebraic(sats)
    c = 2.99792458
    x = [sat[1:3] for sat in sats]
    t = [sat[4] for sat in sats]
    A = hcat([x[i] - x[4] for i = 1:3]...)'
    B = c^2 * (t[1:3] .- t[4])
    sum4 = sum(x[4] .^ 2)
    C = -0.5 * [ c^2*(t[i]^2 - t[4]^2) - sum(x[i] .^ 2) + sum4 for i = 1:3]
    
    a = A \ B
    b = A \ C
    bdiff = b - x[4]
    
    coeffs = Array{Float64}(3)
    coeffs[1] = sum(bdiff .^ 2) - c^2*t[4]^2
    coeffs[2] = 2*(sum(a .* bdiff) + c^2*t[4])
    coeffs[3] = sum(a) - c^2
    guesses = filter(x -> isreal(x), roots(Poly(coeffs)))
    
    location = vcat([a[i]*guesses[1] + b[i] for i = 1:3], guesses[1])
    if(length(guesses) > 1)
        location2 = vcat([a[i]*guesses[2] + b[i] for i = 1:3], guesses[2])
        if(sum(abs2, location2) < sum(abs2, location))
            location = location2
        end
    end
    
    vcat(location[1:3]*10^2, location[4]/10^3)
end;

In [120]:
function prepsat!(x, sat; newton::Bool=true)
    c = 299792.458
    if(!newton)
        c = 2.99792458
    end
    for i = 1:length(sat)
        sat[i][4] = norm(x[1:3]-sat[i][1:3]) / c + x[4]
    end
end

function toalgebraic(sat; onedim::Bool=false)
    newsat = deepcopy(sat)
    if(onedim)
        newsat[4] *= 10^3
        newsat[1:3] /= 10^2
    else 
        for i = 1:length(sat)
            newsat[i][4] *= 10^3
            newsat[i][1:3] /= 10^2
        end
    end
    newsat
end

sat = Array{Array{Float64}}(4)
satalg = Array{Array{Float64}}(4)

sat[1] = [15600.0, 7540.0, 20140.0, 7.074e-2]
sat[2] = [18760.0, 2750.0, 18610.0, 7.220e-2]
sat[3] = [17610.0, 14630.0, 13480.0, 7.690e-2]
sat[4] = [19170.0, 610.0, 18390.0, 7.242e-2]

satalg = toalgebraic(sat)

println(newton(sat))
println(algebraic(satalg))

x = [123.456789, -1234.123, 6300.5392, 5.67e-4]
prepsat!(x, sat)
prepsat!(toalgebraic(x, onedim=true), satalg, newton=false)
println(newton(sat))
println(algebraic(satalg))

x = [12.823234, 3.1231233, -7.12387623, 9.43e-3]
prepsat!(x, sat)
prepsat!(toalgebraic(x, onedim=true), satalg, newton=false)
println(newton(sat))
println(algebraic(satalg))

[-41.7727, -16.7892, 6370.06, -0.00320157]
[-41.7726, -16.7955, 6369.21, -0.00319148]
[123.457, -1234.12, 6300.54, 0.000567]
[123.456, -1234.12, 6300.52, 0.000567278]
[12.8232, 3.12312, -7.12388, 0.00943]
[12.8183, 3.12112, -7.12717, 0.0094324]
